# PyBioPAX tutorial

PyBioPAX implements a native Python object model for BioPAX Level 3. It also provides
multiple API endpoints for processing BioPAX content: from in-memory strings,
local files, or URLs, as well as through graph queries to the Pathway Commons
web service.

All key API functions are exposed at the `pybiopax` package level, while the
BioPAX object model is exposed via the `pybiopax.biopax` module.

In [1]:
import pybiopax

## Processing a local BioPAX OWL file
The `model_from_owl_file` function can be used to process a local BioPAX OWL file.

In [2]:
import os
import pybiopax
local_owl_file = os.path.join(pybiopax.__path__[0], 'tests', 'biopax_test.owl.gz')
model = pybiopax.model_from_owl_gz(local_owl_file)

Processing OWL elements:   0%|          | 0.00/58.0k [00:00<?, ?it/s]

As seen above, PyBioPAX shows a progress bar over the set of OWL elements in
the BioPAX OWL file as they are being processed into Python objects.

## Processing a BioPAX OWL file directly from the web

The `process_from_owl_url` function can be used to process BioPAX OWL from a URL.

In [3]:
url = 'https://raw.githubusercontent.com/indralab/pybiopax/master/pybiopax/tests/molecular_interactions_test.owl'
pybiopax.model_from_owl_url(url)

Processing OWL elements:   0%|          | 0.00/63.0 [00:00<?, ?it/s]

## Querying the Pathway Commons web service
PyBioPAX makes a client available to the Pathway Commons web service in the `pybiopax.pc_client`
module. The function within this module that can be called to execute graph-pattern
queries on the web service is called `graph_query`. It supports three types of
queries:
- `neighborhood`: returns a model around a given source entity
- `paths-between`: given a list of entities, it returns a model of paths between these entities
- `paths-from-to`: given one or more source entities and target entities, it returns a model of paths from the sources to the targets.

In the example below, we query for the neighborhood of the ATF4 gene.

In [4]:
owl_str = pybiopax.pc_client.graph_query(kind='neighborhood', source='ATF4')

In [5]:
model = pybiopax.model_from_owl_str(owl_str)

Processing OWL elements:   0%|          | 0.00/63.8k [00:00<?, ?it/s]

## Traversing the BioPAX model
### Iterating over all objects in the model
BioPAX objects in a model are stored in the `model.objects` dict, whose keys are the
objects' UIDs and values are the objects themselves. This allows quickly
referencing BioPAX objects by UID.

As an example, below we iterate over all the objects in a model, and find
the 10 most common types of objects in the model using a Counter.

In [6]:
from collections import Counter
Counter([obj.__class__.__name__  for uid, obj in model.objects.items()]).most_common(10)

[('RelationshipXref', 11677),
 ('Control', 8734),
 ('ModificationFeature', 7782),
 ('SequenceSite', 6453),
 ('PublicationXref', 6119),
 ('UnificationXref', 4573),
 ('TemplateReactionRegulation', 2905),
 ('Protein', 2362),
 ('SmallMolecule', 2205),
 ('SmallMoleculeReference', 2187)]

### Iterating over certain types of objects in the model

It is often useful to create a loop over all of the objects of a given
type in the model, for instance, to iterate over all `BiochemicalReaction`s,
all `Control`s, or all `PhysicalEntity` objects.

In the example below, we iterate over all `BiochemicalReaction`s and print
their left and right hand sides to get a superficial sense
of what each reaction does (one generally needs to traverse the two sides more
deeply, referencing features, etc. to fully capture each reaction).

In [7]:
for reaction in model.get_objects_by_type(pybiopax.biopax.BiochemicalReaction):
    print('%s -> %s' % (reaction.left, reaction.right))

[Protein(ATF2)] -> [Protein(ATF2)]
[Dna(HERPUD1 gene)] -> [Protein(HERPUD1)]
[Protein(JDP2)] -> [Protein(JDP2)]
[SmallMolecule(N-methyl-7H-dibenzo(c,g)carbazole), Rna(HPRT1 gene)] -> [Rna(HPRT1 gene)]
[Dna(IGFPB1 gene)] -> [Protein(IGFBP1)]
[Protein(CASP9 protein)] -> [Protein(CASP9 protein)]
[Complex(dAP1)] -> [Complex(dAP1)]
[Protein(ATF5)] -> [Protein(ATF5)]
[Protein(ATF-4)] -> [Protein(ATF-4)]
[Rna(ATF4 mRNA)] -> [Protein(ATF4)]
[Protein(ATF-4)] -> [Protein(ATF-4)]
[Protein(PTK2 protein)] -> [Protein(PTK2 protein)]
[Protein(JUN)] -> [Protein(JUN)]
[Protein(ATF7)] -> [Protein(ATF7)]
[Protein(FOS)] -> [Protein(FOS)]
[Protein(ATF-4)] -> [Protein(ATF-4)]
[Dna(IL8 gene)] -> [Protein(IL8)]
[Protein(ATF3)] -> [Protein(ATF3)]
[Protein(ATF-4)] -> [Protein(ATF-4)]
[Protein(ATF-4)] -> [Protein(ATF-4)]
[Rna(TK1 gene)] -> [Rna(TK1 gene)]
[Protein(ATF6)] -> [Protein(ATF6)]
[Protein(ATF4)] -> [Protein(ATF4)]
[Protein(CASP3 protein)] -> [Protein(CASP3 protein)]
[Protein(ATF)] -> [Protein(ATF)]
[Pr

### Referencing a specific object by UID

It is often useful to refer directly to an object by its UID. This can be done by indexing
directly into the `model.objects` attribute, which is a dict keyed by object UIDs.

In the example below, we get a reaction object whose identifier is
`http://identifiers.org/reactome/R-HSA-381128`, and then reference its `left` attribute.

In [8]:
model.objects['http://identifiers.org/reactome/R-HSA-381128'].left

[Rna(ATF4 mRNA)]

### Example: Find all the phosphorylation sites of all proteins in a given model
In this example, we query the Pathway Commons web service for the neighborhood of EGFR
and then traverse the model to find all the phosphorylation sites associated
with proteins in the model.

In [9]:
owl_str = pybiopax.pc_client.graph_query(kind='neighborhood', source='EGFR')
model = pybiopax.model_from_owl_str(owl_str)

Processing OWL elements:   0%|          | 0.00/215k [00:00<?, ?it/s]

In [10]:
# We will capture the set of phosphosites in a dict keyed by protein
from collections import defaultdict
phosphosites = defaultdict(set)

# We iterate over all Protein objects in the model
for protein in model.get_objects_by_type(pybiopax.biopax.Protein):
    # We use the Protein's entity reference's display name for indexing
    name = protein.entity_reference.display_name
    # Iterating over all the protein's features
    for feature in protein.feature:
        # If this is a modification feature which has a known type
        # and that type includes "phospho", i.e., is a phosphorylation
        if isinstance(feature, pybiopax.biopax.ModificationFeature):
            if feature.modification_type and \
                    any('phospho' in mod for mod
                        in feature.modification_type.term):
                # If the site has a location provided
                if feature.feature_location and \
                        isinstance(feature.feature_location, pybiopax.biopax.SequenceSite) and \
                        feature.feature_location.sequence_position is not None:
                    phosphosites[name].add(feature.feature_location.sequence_position)
                    

We now list the posphosites of the top 5 proteins with most sites in this model

In [11]:
for protein, sites in sorted(phosphosites.items(),
                             key=lambda x: len(x[1]),
                             reverse=True)[:5]:
    print(protein, ', '.join(sorted(sites, key=lambda x: int(x))))

EGFR_HUMAN 229, 654, 678, 693, 727, 764, 768, 845, 869, 892, 915, 925, 944, 991, 992, 998, 1016, 1026, 1039, 1041, 1045, 1046, 1064, 1068, 1069, 1070, 1071, 1081, 1085, 1086, 1092, 1096, 1101, 1104, 1110, 1120, 1125, 1138, 1148, 1166, 1172, 1173, 1197
FGFR2_HUMAN 464, 465, 466, 467, 584, 585, 586, 587, 588, 589, 654, 655, 656, 657, 658, 731, 732, 733, 734, 767, 768, 769, 770, 778, 779, 780
GAB1_HUMAN 285, 373, 406, 446, 447, 472, 527, 589, 619, 627, 657, 659, 689
JAK2_HUMAN 119, 221, 372, 373, 570, 637, 813, 868, 966, 972, 1007, 1008
FGFR3_HUMAN 577, 579, 647, 648, 649, 650, 724, 726, 760, 762, 770, 772


## Saving a BioPAX model

BioPAX models can be serialized back to OWL strings or files using the
`pybiopax.model_to_owl_str` or `pybiopax.model_to_owl_file` functions, respectively.

In the example below, we serialize a model into a file called `model.owl`, and show its
first 25 lines.

In [12]:
pybiopax.model_to_owl_file(model, 'model.owl')

Serializing OWL elements:   0%|          | 0/215405 [00:00<?, ?it/s]

In [13]:
with open('model.owl', 'r') as fh:
    head = ''.join([fh.readline() for _ in range(25)])
    print(head)

<?xml version='1.0' encoding='utf-8'?>
<rdf:RDF xmlns:xsd="http://www.w3.org/2001/XMLSchema#" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bp="http://www.biopax.org/release/biopax-level3.owl#" xml:base="http://pathwaycommons.org/pc12/">
  <owl:Ontology rdf:about="">
 <owl:imports rdf:resource="http://www.biopax.org/release/biopax-level3.owl#"/>
  </owl:Ontology>

<bp:SequenceSite rdf:ID="SequenceSite_bfd0338a029352b8e0668f3eb0f6bbac">
 <bp:positionStatus rdf:datatype="http://www.w3.org/2001/XMLSchema#string">EQUAL</bp:positionStatus>
 <bp:sequencePosition rdf:datatype="http://www.w3.org/2001/XMLSchema#int">272</bp:sequencePosition>
</bp:SequenceSite>

<bp:SequenceSite rdf:ID="SequenceSite_14f64bc4cd23eca47d429e39069e06c7">
 <bp:positionStatus rdf:datatype="http://www.w3.org/2001/XMLSchema#string">EQUAL</bp:positionStatus>
 <bp:sequencePosition rdf:datatype="http://www.w3.org/2001/XMLSchema#int">613</bp:sequencePosition>
</bp:S

## Using path constraint strings to traverse a model
Using the `pybiopax.paths` module we can find lists of objects that satisfy a path constraint starting from a given object.

In [14]:
from pybiopax.paths import find_objects

Let's start with a modification feature objects as a starting point for traversal.

In [15]:
modf = model.objects['ModificationFeature_23b018cf79493a97029da4309b044958']
modf

ModificationFeature(SequenceModificationVocabulary("optyr", "O4'-phospho-L-tyrosine")@SequenceSite(628))

Let's now link to the entities of which this is a modification feature using the `feature_of` link. This returns a list of physical entities, in this example, just one.

In [16]:
find_objects(modf, 'feature_of')

[Protein(IL3RB)]

We can extend the path constraint to then link forward to all other features of the entities that this is a feature of using the `feature` link. We additionally set a class constraint `ModificationFeature` to return only `ModificationFeature`s. Finally, we link to the type of the modification feature through the `modification_type` link.

In [17]:
find_objects(modf, 'feature_of/feature:ModificationFeature/modification_type')

[SequenceModificationVocabulary("optyr", "O4'-phospho-L-tyrosine"),
 SequenceModificationVocabulary("optyr", "O4'-phospho-L-tyrosine")]